## To What Extent Does Weather Impact Percent On Time of Buses?
   This notebook is intended to be used to take MTBA bus data and weather data collected by a NOAA station in boston to determine if weather has a measurable impact on the percent of buses that are on time. The MTBA data set includes for its bus entries if it's reccord is from peak hours or off peak as well as number of busses on time that day for that route and number of total busses that ran on that route that day. The weather data includes a number of attributes not as useful, but does include inches of snow, rain, as well as Average Temperature, Max Temperature, and Min Temperature. Both datasets entries have dates, which can and will be joined on to get the data sets together. After which the data will be reorganized for use by a machine learning algorithm or two. The goal is to find an algorithm that will produce a model with good performance, and then look at the weights of different weather to come to a conclusion on how impactful weather is for percent on time.  

In [1]:
import pandas as pd
import numpy as np

#### These first few cells are for combining the weather and bus data
I ended up joining the two sets on date and then reducing the columns. The bus data had two columns that could be useful for this which were route number and peak or off peak. I initially removed route number but might try to bring it back in. The peak or off peak attribute I figured would be interesting but discovered it was making all of the models perform very poorly. 

In [10]:
bus = pd.read_csv("bus_forLearning.csv")
weath = pd.read_csv("weather_trimmed.csv")
bus.tail()


,SERVICE_DATE,peak/offpeak,ROUTE_OR_LINE,PERCENT_ONTIME
198185,3/4/2018,0,45,48
198186,3/4/2018,0,99,58
198187,3/4/2018,0,109,51
198188,3/4/2018,0,70,64
198189,3/4/2018,0,95,67


#### Above reads in the two datasets, already prepared to be joined, below joins data sets and outputs for further trimming and re-ordering

In [11]:
bw = bus.join(weath.set_index('DATE'), on ='SERVICE_DATE') #join datasets on dates
#bw = bw.drop( columns = ['SERVICE_DATE','PEAK_OFFPEAK_IND']) # date no longer needed, may confuse algorithms
bw.tail()
#bw.to_csv("busWithWeather1.csv")

,SERVICE_DATE,peak/offpeak,ROUTE_OR_LINE,PERCENT_ONTIME,PRCP,SNOW,TAVG,TMAX,TMIN
198185,3/4/2018,0,45,48,0.0,0.0,39.0,41.0,36.0
198186,3/4/2018,0,99,58,0.0,0.0,39.0,41.0,36.0
198187,3/4/2018,0,109,51,0.0,0.0,39.0,41.0,36.0
198188,3/4/2018,0,70,64,0.0,0.0,39.0,41.0,36.0
198189,3/4/2018,0,95,67,0.0,0.0,39.0,41.0,36.0


#### Read in resulting data that's prepared for learning. Split data into training and test partitions after random re-ordering of entries

In [57]:
learn = pd.read_csv("bw_cleaned.csv", header = None)
#learn.info()

learn = learn.sample(frac = 1)
train_x = learn.iloc[0:120001,0:5]
train_y = learn.iloc[0:120001, 6]

test_x = learn.iloc[120002:,0:5]
test_y = learn.iloc[120002:, 6]



### The above data includes the "peak or off peak hours" attribute, the bottom does not.

In [3]:
learn2 = pd.read_csv("bw_nopeak.csv",header = None)

learn2 = learn2.sample(frac = 1 )
train_x = learn2.iloc[0:180001,0:4]
train_y = learn2.iloc[0:180001,5]

test_x = learn2.iloc[180002:,0:4]
test_y = learn2.iloc[180002:,5]

#print(train_y)


In [5]:
from sklearn.neural_network import MLPRegressor as mlp
from sklearn.linear_model import TheilSenRegressor as TSR, SGDRegressor as SGD, LogisticRegression as LR
from sklearn.svm import LinearSVR as LSVR, SVR
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.metrics import mean_absolute_error as mae, r2_score as r2, explained_variance_score as evs

In [11]:
rfr = RFR()
rfr.fit(train_x,train_y)
training_mae = mae(train_y, rfr.predict(train_x)) #best is 0
training_r2 = r2(train_y, rfr.predict(train_x)) #best 1, can be neg(opposite of helpful good)
training_evs = evs(train_y, rfr.predict(train_x)) #best is 1 , lower worse
print("training :: mae: %0.6f, r2: %0.6f, evs: %0.6f" % (training_mae,training_r2,training_evs))
#print("best i %d" % best_i)

test_mae = mae(test_y, rfr.predict(test_x))
test_r2 = r2(test_y, rfr.predict(test_x))
test_evs = evs(test_y, rfr.predict(test_x))
print("testing :: mae: %0.6f, r2: %0.6f, evs: %0.6f" % (test_mae,test_r2,test_evs))

training :: mae: 12.678117, r2: 0.041851, evs: 0.041853
testing :: mae: 12.738307, r2: 0.041537, evs: 0.041556


In [12]:
## FOR RANDOM FOREST REGRESSOR -> REALLY JUST DESCISION TREE
best_mae = 20
best_r2 = 0
best_evs = 0
best_i = 0
best_rfr = RFR()
for i in range(1,20):
    rfr = RFR(n_estimators = 10, max_depth = 12, min_samples_leaf = i)
    rfr.fit(train_x,train_y)
    training_mae = mae(train_y, rfr.predict(train_x)) #best is 0
    training_r2 = r2(train_y, rfr.predict(train_x)) #best 1, can be neg(opposite of helpful good)
    training_evs = evs(train_y, rfr.predict(train_x)) #best is 1 , lower worse
    if((best_mae>training_mae)&(best_r2<training_r2)&(best_evs<training_evs)):
        best_mae = training_mae
        best_r2 = training_r2
        best_evs = training_evs
        best_i = i
        best_rfr = rfr
print("training :: mae: %0.6f, r2: %0.6f, evs: %0.6f" % (best_mae,best_r2,best_evs))
print("best i %d" % best_i)

test_mae = mae(test_y, best_rfr.predict(test_x))
test_r2 = r2(test_y, best_rfr.predict(test_x))
test_evs = evs(test_y, best_rfr.predict(test_x))
print("testing :: mae: %0.6f, r2: %0.6f, evs: %0.6f" % (test_mae,test_r2,test_evs))

training :: mae: 12.684855, r2: 0.040723, evs: 0.040723
best i 6
testing :: mae: 12.752932, r2: 0.039421, evs: 0.039453


In [51]:
print("feature weights", best_rfr.feature_importances_)
print("n_feat ", best_rfr.n_features_)
print("n_outputs ", best_rfr.n_outputs_)
#print("oob_score ", best_rfr.oob_score_)
#best_rfr.decision_path(train_x)

feature weights [0.24552306 0.14108116 0.02043798 0.34368151 0.24927629]
n_feat  5
n_outputs  1


In [ ]:
## for k cross fold validation of rfr
from sklearn.model_selection import RepeatedKFold
x = learn.iloc[0:,0:4]
y = learn.iloc[0:,5]



In [9]:
## FOR MULITLAYER PERCEPTRON NEURAL NETWORK
mlpr = mlp()
mlpr.fit(train_x,train_y)
training_mae = mae(train_y, mlpr.predict(train_x)) #best is 0
training_r2 = r2(train_y, mlpr.predict(train_x)) #best 1, can be neg(opposite of helpful good)
training_evs = evs(train_y, mlpr.predict(train_x)) #best is 1 , lower worse
print("training :: mae: %0.6f, r2: %0.6f, evs: %0.6f" % (training_mae,training_r2,training_evs))

#after having removed peak/off peak flag from data set training performance has gone up
test_mae = mae(test_y, mlpr.predict(test_x))
test_r2 = r2(test_y, mlpr.predict(test_x))
test_evs = evs(test_y, mlpr.predict(test_x))
print("testing :: mae: %0.6f, r2: %0.6f, evs: %0.6f" % (test_mae,test_r2,test_evs))


training :: mae: 13.815569, r2: -0.061698, evs: 0.007538
testing :: mae: 13.924743, r2: -0.071045, evs: 0.006825


In [13]:
## FOR THEILSENREGRESSOR 
tsr = TSR()
tsr.fit(train_x,train_y)
training_mae = mae(train_y, tsr.predict(train_x)) #best is 0
training_r2 = r2(train_y, tsr.predict(train_x)) #best 1, can be neg(opposite of helpful good)
training_evs = evs(train_y, tsr.predict(train_x)) #best is 1 , lower worse
print("mae: %0.6f, r2: %0.6f, evs: %0.6f" % (training_mae,training_r2,training_evs))

## Have a positive r2 value for this regressor, so it might be worth tuning

test_mae = mae(test_y, tsr.predict(test_x))
test_r2 = r2(test_y, tsr.predict(test_x))
test_evs = evs(test_y, tsr.predict(test_x))
print("testing :: mae: %0.6f, r2: %0.6f, evs: %0.6f" % (test_mae,test_r2,test_evs))

mae: 12.856826, r2: 0.001557, evs: 0.006583
testing :: mae: 12.903749, r2: 0.001594, evs: 0.007458


In [59]:
## FOR STOCHASTIC GRADIENT DESCENT
sgd = SGD(max_iter = 1000)
sgd.fit(train_x,train_y)
training_mae = mae(train_y, sgd.predict(train_x)) #best is 0
training_r2 = r2(train_y, sgd.predict(train_x)) #best 1, can be neg(opposite of helpful good)
training_evs = evs(train_y, sgd.predict(train_x)) #best is 1 , lower worse
print("mae: %0.6f, r2: %0.6f, evs: %0.6f" % (training_mae,training_r2,training_evs))

#did much worse than mlp. probably not useful
test_mae = mae(test_y, sgd.predict(test_x))
test_r2 = r2(test_y, sgd.predict(test_x))
test_evs = evs(test_y, sgd.predict(test_x))
print("testing :: mae: %0.6f, r2: %0.6f, evs: %0.6f" % (test_mae,test_r2,test_evs))

mae: 3.070260, r2: 0.934703, evs: 0.956831
testing :: mae: 3.067188, r2: 0.935070, evs: 0.957273


In [66]:
## FOR LOGISTIC REGRESSION

lr = LR()
lr.fit(train_x,train_y)
training_mae = mae(train_y, lr.predict(train_x)) #best is 0
training_r2 = r2(train_y, lr.predict(train_x)) #best 1, can be neg(opposite of helpful good)
training_evs = evs(train_y, lr.predict(train_x)) #best is 1 , lower worse
print("mae: %0.6f, r2: %0.6f, evs: %0.6f" % (training_mae,training_r2,training_evs))

test_mae = mae(test_y, lr.predict(test_x))
test_r2 = r2(test_y, lr.predict(test_x))
test_evs = evs(test_y, lr.predict(test_x))
print("testing :: mae: %0.6f, r2: %0.6f, evs: %0.6f" % (test_mae,test_r2,test_evs))

mae: 5.100791, r2: 0.826813, evs: 0.829544
testing :: mae: 5.101414, r2: 0.826410, evs: 0.829181


In [65]:
## FOR SVM
svm = SVR(kernel = 'rbf',max_iter = 1000)
svm.fit(train_x,train_y)
training_mae = mae(train_y, svm.predict(train_x)) #best is 0
training_r2 = r2(train_y, svm.predict(train_x)) #best 1, can be neg(opposite of helpful good)
training_evs = evs(train_y, svm.predict(train_x)) #best is 1 , lower worse
print("mae: %0.6f, r2: %0.6f, evs: %0.6f" % (training_mae,training_r2,training_evs))

test_mae = mae(test_y, svm.predict(test_x)) ####this one has under performed with the new data, not going to be useful
test_r2 = r2(test_y, svm.predict(test_x))
test_evs = evs(test_y, svm.predict(test_x))
print("testing :: mae: %0.6f, r2: %0.6f, evs: %0.6f" % (test_mae,test_r2,test_evs))

C:\Users\Theo\Anaconda3\lib\site-packages\sklearn\svm\base.py:218: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


mae: 14.797663, r2: -0.175069, evs: 0.019918
testing :: mae: 14.771081, r2: -0.177553, evs: 0.020015


In [ ]:
## others -> beta regression, nearest neighbors?